#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [39]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop, SGD
from kerastuner.tuners import BayesianOptimization
from keras import backend as K

In [40]:
final_dataset = pd.read_csv('../../Datasets/final_dataset_by_client.csv', sep='\t')

In [41]:
final_dataset.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0


In [42]:
# columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'age', 'y']
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']
n_features = 5

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos desde i-0 hasta i-n, y el valor de "y" sera el valor de "y" 2 meses en el futuro ( i+2 ).

In [43]:
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix >= len(data):
            break
        seq_x, seq_y = data[:i+1, :], data[end_ix, -1]  # y es 2 periodos en el futuro
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X, dtype=object), np.array(y)


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [44]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [45]:
def build_model(hp):
    activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    units = hp.Int('units', min_value=32, max_value=256, step=32)
    dropout = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    l2_penalty = hp.Choice('l2_penalty', values=[1e-2, 1e-3, 1e-4])
    depth = hp.Int('depth', min_value=1, max_value=5)
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    model = Sequential()
    model.add(LSTM(units=int(units/2), activation=activation, input_shape=(None, n_features), return_sequences=True, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    for _ in range(depth - 1):
        model.add(LSTM(units=units, activation=activation, return_sequences=True, kernel_regularizer=l2(l2_penalty)))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    
    model.add(LSTM(units=int(units*2), activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='relu'))

    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='mse')

    return model

#### Armado de los modelos

In [46]:
# # Codigo para visualizar como queda la estructura de X e y para 1 producto en particular
# product_ids = final_dataset['product_id'].unique()
# product_id =  product_ids[0]
# product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[columns]
# product_data_array = product_data.values
# X, y = prepare_data(product_data_array, n_steps)

# display(product_data)
# display(X)

In [47]:
# TODO: ADD SCALER
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

In [48]:
# # Calculo el promedio de re-compra para aquellos productos que se comprar 2 meses solamente para un cliente en particular.
# # Esto me va a servir para la combinacion de producto_id y customer_id que solamente tengo 2 registros o 1.
# # Lo hago sobre estos casos y no por todos los product_id/customer_id, porque no tengo todo el historia de venta completo
# # para todos los clientes => no puedo garantizar que mi mes1 y mes2 para algunos clientes sean su primer y segunda compra.

# product_customer_ids = final_dataset[['product_id', 'customer_id']].drop_duplicates().values
# sum = 0
# mes1 = 0
# mes2 = 0
    
# for product_id, customer_id in product_customer_ids:

#     product_data = final_dataset[(final_dataset['product_id'] == product_id) & (final_dataset['customer_id'] == customer_id)].sort_values(by='periodo')[columns]
    
#     if product_data.shape[0] == 2:
#         mes1 += product_data.iloc[-2]['y']
#         mes2 += product_data.iloc[-1]['y']
#         sum += 1
        
# print(((mes2 - mes1) / mes1) * 100)

# Resultado = 4.556482896396117, comentado porque tarda 1hs aprox en ejecutar

In [52]:
import os
import random


product_customer_ids = final_dataset[['product_id', 'customer_id']].drop_duplicates().values
    
for product_id, customer_id in product_customer_ids[:5]:
    
    tuner = BayesianOptimization(
        build_model,
        objective='val_loss',
        max_trials=5, # Decrementar a 2 para una prueba "rapida" al validar algun cambio
        executions_per_trial=2,
        directory='bayesian_optimization',
        project_name=f'lstm_hyperparameters_{product_id}')
    
    # Modifico la semilla en cada iteracion, sino genera 1 modelo solo y luego predice siempre con el mismo.
    # Asi me aseguro que arme 1 modelo por cada producto, osea, que se vuevla a ejecutar la bayesiana.
    seed = np.random.randint(0, 10000)
    random.seed(seed)

    product_data = final_dataset[(final_dataset['product_id'] == product_id) & (final_dataset['customer_id'] == customer_id)].sort_values(by='periodo')[columns]
    
    # Convertir los datos a numpy array
    product_data_array = product_data.values
    
    # Verificar si el tamaño de product_data_array es menor que 3, esto quiere decir que tengo menos
    # de 3 ventas registradas para ese product_id / customer_id
    if len(product_data_array) < 3:
        last_y = product_data_array[-1, -1] * 1.05 # Usar el valor de "y" del último registro multiplicado por 1.05 como predicción por la celda anteior
        predictions.append({'product_id': product_id, 'customer_id': customer_id, 'predicted_y': last_y})
        print(f'No se generó modelo para el producto {product_id} y cliente {customer_id}. Predicción a 2 meses: {last_y}')
        continue
    
    # Preparar los datos para LSTM
    X, y = prepare_data(product_data_array, n_steps)

    # Callback para detener el entrenamiento temprano
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=10, min_lr=0.0001)

    max_length = max(len(seq) for seq in X)
    X_padded = np.array([np.pad(seq, ((max_length - len(seq), 0), (0, 0)), 'constant') for seq in X]) # Pad left para que cada registro tenga la misma longitud
    
    # Convertir X a una lista de arrays para que funcione con keras tuner
    X_list = [x.tolist() for x in X_padded]

    tuner.search(np.array(X_list), y, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stopping, reduce_lr], verbose = 0 )
    best_model = tuner.get_best_models(num_models=1)[0]

    # Guardar el modelo
    os.makedirs('Models_params', exist_ok=True)
    best_model.save(f'Models_params/model_product_{product_id}_customer_{customer_id}.h5')

    last_record = product_data_array
    last_record = last_record.reshape((1, last_record.shape[0], last_record.shape[1]))
    predicted_y = best_model.predict(last_record)[0][0]

    # Agregar predicción al resultado
    predictions.append({'product_id': product_id, 'customer_id': customer_id, 'predicted_y': predicted_y})

    print(f'Modelo para el producto {product_id} y cliente {customer_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}')

1/1 [==============================] - 1s 1s/step
Modelo para el producto 20001 y cliente 10001 entrenado y guardado. Predicción a 2 meses: 417.0064697265625
Reloading Tuner from bayesian_optimization/lstm_hyperparameters_20001/tuner0.json
1/1 [==============================] - 1s 925ms/step
Modelo para el producto 20001 y cliente 10002 entrenado y guardado. Predicción a 2 meses: 164.10000610351562
Reloading Tuner from bayesian_optimization/lstm_hyperparameters_20001/tuner0.json
1/1 [==============================] - 1s 945ms/step
Modelo para el producto 20001 y cliente 10003 entrenado y guardado. Predicción a 2 meses: 291.5449523925781
Reloading Tuner from bayesian_optimization/lstm_hyperparameters_20001/tuner0.json


KeyboardInterrupt: 

In [50]:
predictions_df = pd.DataFrame(predictions)
final_predictions_df = predictions_df.groupby('product_id')['predicted_y'].sum().reset_index()

final_predictions_df.to_csv('../../Datasets/predictions_lstm_productos_clientes_bayesiana.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions_lstm_productos_clientes_bayesiana.csv.')

Todas las predicciones han sido generadas y guardadas en predictions_lstm_productos_clientes_bayesiana.csv.
